# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient('127.0.0.1', 27017) # Si utilisation en local
# client = pymongo.MongoClient('mongo') # Si utilisation d'une image mongo pour Docker
database = client['exercices']
collection_yt = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [4]:
yt_json = pd.read_json("./data/US_category_id.json")
yt_json.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


## Question 0

### Netoyer les données

In [5]:
from pandas.io.json import json_normalize

lst_drop = ['kind', 'snippet.channelId', 'etag']

def traitement_json(df, lst_drop):
    df_norm = pd.json_normalize(df['items'])
    df_norm = df_norm.drop(columns = lst_drop)
    df_norm['id'] = df_norm['id'].astype(int)
    
    return df_norm

category_json = traitement_json(yt_json, lst_drop)
category_json = category_json.rename(columns = {'snippet.title':'snippet_title', 'snippet.assignable':'snippet_assignable'})
category_json.head()

,id,snippet_title,snippet_assignable
0,1,Film & Animation,True
1,2,Autos & Vehicles,True
2,10,Music,True
3,15,Pets & Animals,True
4,17,Sports,True


In [6]:
category_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  32 non-null     int64 
 1   snippet_title       32 non-null     object
 2   snippet_assignable  32 non-null     bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 672.0+ bytes


In [7]:
df_youtube.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7992 entries, 0 to 7991
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        7992 non-null   object 
 1   title           7992 non-null   object 
 2   channel_title   7992 non-null   object 
 3   category_id     7992 non-null   int64  
 4   tags            7992 non-null   object 
 5   views           7992 non-null   int64  
 6   likes           7992 non-null   int64  
 7   dislikes        7992 non-null   int64  
 8   comment_total   7992 non-null   int64  
 9   thumbnail_link  7992 non-null   object 
 10  date            7992 non-null   float64
dtypes: float64(1), int64(5), object(5)
memory usage: 686.9+ KB


In [8]:
df_yt = df_youtube.merge(category_json, left_on = 'category_id', right_on = 'id')
df_yt = df_yt.drop(columns = 'id')

In [9]:
df_yt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7992 entries, 0 to 7991
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   video_id            7992 non-null   object 
 1   title               7992 non-null   object 
 2   channel_title       7992 non-null   object 
 3   category_id         7992 non-null   int64  
 4   tags                7992 non-null   object 
 5   views               7992 non-null   int64  
 6   likes               7992 non-null   int64  
 7   dislikes            7992 non-null   int64  
 8   comment_total       7992 non-null   int64  
 9   thumbnail_link      7992 non-null   object 
 10  date                7992 non-null   float64
 11  snippet_title       7992 non-null   object 
 12  snippet_assignable  7992 non-null   bool   
dtypes: bool(1), float64(1), int64(5), object(6)
memory usage: 819.5+ KB


In [10]:
df_yt

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,snippet_title,snippet_assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,Entertainment,True
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,mtv|video|online|official|tv|television|watch|...,274358,9215,477,838,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,13.09,Entertainment,True
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,24,America's Got Talent 2017|america's got talent...,473691,14740,415,1696,https://i.ytimg.com/vi/vu_9muoxT50/default.jpg,13.09,Entertainment,True
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,24,panda|what should we eat|buzzfeed|comedy|boyfr...,282858,14870,300,1398,https://i.ytimg.com/vi/ZQK1F0wz6z4/default.jpg,13.09,Entertainment,True
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,24,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,13.09,Entertainment,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7987,ThPzNokG5uE,Japanese Candy Art,INSIDER,19,Shinobu Ichiyanagi|Japanese|Candy|Art|Amezaiku...,262878,7594,62,663,https://i.ytimg.com/vi/ThPzNokG5uE/default.jpg,19.10,Travel & Events,True
7988,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,383835,10457,275,1948,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,19.10,Travel & Events,True
7989,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,396811,10633,279,1552,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,20.10,Travel & Events,True
7990,5mdk5oz6l3w,Pixel 2: Squeeze to access features (CNET News),CNET,43,Google|Pixel 2|Squeeze|Google Assistant|Phones...,8259,111,8,37,https://i.ytimg.com/vi/5mdk5oz6l3w/default.jpg,6.10,Shows,False


In [11]:
df_yt.drop_duplicates(keep = 'first', inplace=True)

In [12]:
df_yt

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,snippet_title,snippet_assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,Entertainment,True
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,mtv|video|online|official|tv|television|watch|...,274358,9215,477,838,https://i.ytimg.com/vi/4MkC65emkG4/default.jpg,13.09,Entertainment,True
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,24,America's Got Talent 2017|america's got talent...,473691,14740,415,1696,https://i.ytimg.com/vi/vu_9muoxT50/default.jpg,13.09,Entertainment,True
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,24,panda|what should we eat|buzzfeed|comedy|boyfr...,282858,14870,300,1398,https://i.ytimg.com/vi/ZQK1F0wz6z4/default.jpg,13.09,Entertainment,True
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,24,TMZ2016FS11221|TMZ|Hollywood|Celebrity|Enterta...,703750,2921,2196,1042,https://i.ytimg.com/vi/5ywKal6-anc/default.jpg,13.09,Entertainment,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7987,ThPzNokG5uE,Japanese Candy Art,INSIDER,19,Shinobu Ichiyanagi|Japanese|Candy|Art|Amezaiku...,262878,7594,62,663,https://i.ytimg.com/vi/ThPzNokG5uE/default.jpg,19.10,Travel & Events,True
7988,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,383835,10457,275,1948,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,19.10,Travel & Events,True
7989,18HD7Y3tT9o,"DEEP Chinese Street Food Tour in Sichuan, Chin...",The Food Ranger,19,the food ranger|trevor james|food ranger|food|...,396811,10633,279,1552,https://i.ytimg.com/vi/18HD7Y3tT9o/default.jpg,20.10,Travel & Events,True
7990,5mdk5oz6l3w,Pixel 2: Squeeze to access features (CNET News),CNET,43,Google|Pixel 2|Squeeze|Google Assistant|Phones...,8259,111,8,37,https://i.ytimg.com/vi/5mdk5oz6l3w/default.jpg,6.10,Shows,False


In [13]:
df_yt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7992 entries, 0 to 7991
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   video_id            7992 non-null   object 
 1   title               7992 non-null   object 
 2   channel_title       7992 non-null   object 
 3   category_id         7992 non-null   int64  
 4   tags                7992 non-null   object 
 5   views               7992 non-null   int64  
 6   likes               7992 non-null   int64  
 7   dislikes            7992 non-null   int64  
 8   comment_total       7992 non-null   int64  
 9   thumbnail_link      7992 non-null   object 
 10  date                7992 non-null   float64
 11  snippet_title       7992 non-null   object 
 12  snippet_assignable  7992 non-null   bool   
dtypes: bool(1), float64(1), int64(5), object(6)
memory usage: 819.5+ KB


### Importer les données

In [14]:
df_yt_dict = df_yt.to_dict('records')

In [15]:
df_yt_dict

[{'video_id': 'XpVt6Z1Gjjo',
  'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
  'channel_title': 'Logan Paul Vlogs',
  'category_id': 24,
  'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
  'views': 4394029,
  'likes': 320053,
  'dislikes': 5931,
  'comment_total': 46245,
  'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
  'date': 13.09,
  'snippet_title': 'Entertainment',
  'snippet_assignable': True},
 {'video_id': '4MkC65emkG4',
  'title': 'Hand In Hand A Benefit For Hurricane Relief | MTV',
  'channel_title': 'MTV',
  'category_id': 24,
  'tags': "mtv|video|online|official|tv|television|watch|Hand in Hand|benefit|hu

In [16]:
collection_yt.drop()

In [17]:
collection_yt.insert_many(df_yt_dict)

## Question 1  

In [18]:
cur_ex1 = collection_yt.find({'channel_title' : 'Apple'})

In [19]:
list(cur_ex1)

[{'_id': ObjectId('601d0ba534fe727d22f1da30'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09,
  'snippet_title': 'Science & Technology',
  'snippet_assignable': True},
 {'_id': ObjectId('601d0ba534fe727d22f1da40'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 12200526,
  'likes': 258842,
  'dislikes': 44339,
  'comment_total': 0,
  'thumbnail_link': 'https:/

## Question 2

In [20]:
cur_ex2 = collection_yt.aggregate([{"$group" : {"_id" : "$category_id", "nbr_of_category" : {"$sum" : 1}}}])
len(list(cur_ex2))

16

In [21]:
len(collection_yt.distinct('category_id'))

16

## Question 3

In [22]:
cur_ex3 = list(collection_yt.find({}, {"_id":1, "tags":1}))
liste_id_tag = [(d["_id"], d["tags"].split("|")) for d in cur_ex3]
for couple in liste_id_tag:
    _id, tags = couple
    print(couple)
    collection_yt.update_one({'_id' : _id}, {'$set' : {'tags' : tags}})

(ObjectId('601d0ba534fe727d22f1d3ef'), ['logan paul vlog', 'logan paul', 'logan', 'paul', 'olympics', 'logan paul youtube', 'vlog', 'daily', 'comedy', 'hollywood', 'parrot', 'maverick', 'bird', 'maverick clothes', 'diamond play button', 'logan paul diamond play button', '10M subscribers', 'logan paul 1 year vlogging', '1 year vlog', 'dwarf mamba play button', 'logan paul history', 'youtube history', '10M', '10M plaque', 'youtube button', 'diamond button', 'logang', 'logang 4 life'])
(ObjectId('601d0ba534fe727d22f1d3f0'), ['mtv', 'video', 'online', 'official', 'tv', 'television', 'watch', 'Hand in Hand', 'benefit', 'hurricane relief', 'money', 'hurricane harvey', 'hurricane irma', 'beyonce', 'george clooney', 'julia roberts', 'barbra stresiand', 'reese witherspoon', 'oprah winfrey', 'stars', 'celebrity', 'celebrities', 'charity', 'charities', 'aid', 'recovery', 'Houston', 'Florida', 'Miami', 'habitat for humanity', 'save the children', 'feeding texas', "mayor's fund for hurricane harvey

(ObjectId('601d0ba534fe727d22f1d531'), ['Gordon', 'Gordon Ramsay', 'Ramsay', 'Ramsey', 'Chef Ramsay', 'Recipe', 'Recipes', 'Food', 'Cooking', 'Cookery', 'Gordon Ramsey', 'Chef Ramsey', 'Steak', 'How to', 'Gordon Ramsey how to', 'Gordon Ramsay how to', 'home cooking', 'gordon Ramsey home cooking', 'gordon Ramsay home cooking', 'watch gordon Ramsey', 'watch gordon Ramsay', 'eggs', 'hash brown', 'bacon'])
(ObjectId('601d0ba534fe727d22f1d532'), ['tyler oakley', 'tyleroakley', 'youtuber', 'vlog', 'vlogger', 'lgbtq', 'lgbt', 'gay', 'vlogging', 'upload', 'Q&A', 'question', 'answer', 'funny', 'lol', 'cc', 'captioned', '10 year anniversary', 'decade', 'ten year', 'joey graceffa', 'ingrid nilsen', 'alfie deyes', 'caspar lee', 'connor franta', 'gigi gorgeous', 'grace helbig', 'hank green', 'vlog brothers', 'john green', 'jim chapman', 'joe sugg', 'kingsley', 'korey kuhl', 'kyle krieger', 'lilly singh', 'liza koshy', 'mamrie hart', 'zoella'])
(ObjectId('601d0ba534fe727d22f1d533'), ['screen junkies

(ObjectId('601d0ba534fe727d22f1d65e'), ['good morning britain', 'breakfast show', 'news', 'morning news', 'gmb', 'good morning britain interview', 'morning breakfast show', 'itv morning', 'morning show', 'piers morgan', 'susanna reid', 'Steven Seagal', 'Seagal', 'NFL', 'Protest', 'Football', 'Donald Trump', 'President', 'Trump', 'Vladimir', 'Putin', 'Russia', 'US', 'USA'])
(ObjectId('601d0ba534fe727d22f1d65f'), ['What What Happens live', 'reality', 'interview', 'fun', 'celebrity', 'Andy Cohen', 'talk', 'show', 'program', 'Bravo', 'Watch What Happens Live', 'WWHL', 'bravo andy', 'Watch', 'What', 'Happens', 'Carol', 'American', 'American crime story', 'Katrina', 'scrapped', 'revamped', 'hospital', 'Orleans', 'doctor', 'remember', 'euthanized several', 'patients', 'big story', 'Sarah', "Paulsen's", 'playing', 'main doctor', 'room', 'yours truly', 'next year', 'New Orleans', 'Sterling K Brown', 'ACS: Katrina’', 'upcoming revamped', 'Crime Story', 'star actress', 'dishes'])
(ObjectId('601d0

(ObjectId('601d0ba534fe727d22f1d6d2'), ['the voice', 'the voice nbc', 'the voice season 13', 'the voice blind auditions', 'the voice blinds', 'team adam', 'adam levine', 'team cyrus', 'miley cyrus', 'team hudson', 'jenifer hudson', 'team shelton', 'blake shelton', 'dave crosby', 'i will follow you into the dar', 'the voice new season', 'The Voice 2017', 'The Voice USA', 'The Voice Season 13', 'The voice winners', 'jennifer hudson', 'carson daly'])
(ObjectId('601d0ba534fe727d22f1d6d3'), ['elvis duran', 'elvis duran and the morning show', 'elvis duran show', 'elvis duran camila cabello', 'camila cabello performance', 'camila cabello havana', 'camila cabello acoustic'])
(ObjectId('601d0ba534fe727d22f1d6d4'), ['SNL', 'Saturday Night Live', 'SNL Season 43', 'SNL Season 43 Premiere', 'Episode 1726', 'Ryan Gosling', 'Donald Trump', 'Alec Baldwin', 'Jeff Sessions', 'Kate McKinnon', 'Sarah Huckabee Sanders', 'Aidy Bryant', 'Chuck Schumer', 'Alex Moffat', 'Melissa Villasenor', 'Carmen Yulin Cruz

(ObjectId('601d0ba534fe727d22f1d81e'), ['James Corden', 'The Late Late Show', 'Colbert', 'late night', 'late night show', 'Stephen Colbert', 'Comedy', 'monologue', 'comedian', 'impressions', 'celebrities', 'carpool', 'karaoke', 'CBS', 'Late Late Show', 'Corden', 'joke', 'jokes', 'funny', 'funny video', 'funny videos', 'humor', 'celebrity', 'celeb', 'hollywood', 'famous'])
(ObjectId('601d0ba534fe727d22f1d81f'), ['dusk till dawn', 'dusk till dawn zayn', 'dusk till dawn lyrics', 'dusk till dawn cover', 'sia', 'cover', 'audio', 'karaoke', 'instrumental', 'music video', 'kirsten collins', 'blake rose', 'piano', 'kurt hugo schneider'])
(ObjectId('601d0ba534fe727d22f1d820'), ['superfruit', 'super', 'fruit', 'scott', 'hoying', 'mitch', 'grassi', 'fcute', 'weekly', 'obsessions', 'everyone', 'heres', 'good', 'song', 'dont', 'listen', 'to', 'anything', 'else', 'today', 'deny u', 'future friends', 'colin duffy'])
(ObjectId('601d0ba534fe727d22f1d821'), ['your brain vs. your crush', 'your brain vs. 

...

(ObjectId('601d0ba534fe727d22f1ee35'), ['Your World', 'Neil Cavuto', 'Common Sense', 'On Air', 'Opinion', 'Personality', 'Primary Opinion', 'Fox News', 'News'])
(ObjectId('601d0ba534fe727d22f1ee36'), ['vox.com', 'vox', 'explain', 'Rotten Tomatoes', 'film review aggregator', 'film criticism', 'film review', 'Certified Fresh', 'critical consensus', 'Moonlight', 'Get Out', 'Dunkirk', 'Alien Covenant', 'Alissa Wilkinson', 'Rotten Tomatoes explained', 'Rotten Tomatoes controversy', 'Tomatometer', 'Tomatometer rating fair', 'Tomatometer hollywood', '2017 box office slump', 'How Rotten Tomatoes Works', 'Tomatometer rating', 'Rotten Tomatoes tomatometer'])
(ObjectId('601d0ba534fe727d22f1ee37'), ['Business Insider', 'Dove', 'Advertising'])
(ObjectId('601d0ba534fe727d22f1ee38'), ['[none]'])
(ObjectId('601d0ba534fe727d22f1ee39'), ['rap', 'eminem'])
(ObjectId('601d0ba534fe727d22f1ee3a'), ['harvey weinstein', 'fired', 'ousted', 'weinstein company', 'allegations', 'sexual harrassment', 'brian stelte

(ObjectId('601d0ba534fe727d22f1ef44'), ['wwe', 'world wrestling entertainment', 'wrestling', 'wrestler', 'wrestle', 'superstars', 'कुश्ती', 'पहलवान', 'डब्लू डब्लू ई', 'मैच', 'सुपरस्टार', 'व्यावसायिक कुश्ती', 'مصارعه', 'Raw', 'WWE Top 10', 'Finn Bálor', 'Bray Wyatt', 'Elias Samson', 'Apollo Crews', "Titus O'Neil", 'Seth Rollins', 'Bayley', 'The Miz', 'Curtis Axel', 'Bo Dallas', 'matt hardy', 'jason jordan', 'braun strowman', 'mickie james', 'alexa bliss', 'neville', 'enzo amore', 'curt hawkins', 'roman reigns', 'raw clips', 'best of raw'])
(ObjectId('601d0ba534fe727d22f1ef45'), ['dude perfect', 'dude perfect stereotypes', 'dude perfect water bottle flip', 'bottle flip', 'water bottle flip', 'dude perfect bottle flip', 'dude perfect basketball', 'dp', 'dude perfect world record', 'edition', 'nerf', 'trick shots', 'trick shot', 'family', 'ping pong', 'bowling', 'clean', 'family friendly', 'bubble wrap', 'soccer', 'football', 'spinner', 'spinners', 'fidget spinners', 'dude', 'chelsea', 'fc

(ObjectId('601d0ba534fe727d22f1f097'), ['name', 'most popular name', 'names', 'most popular baby names', 'boy names', 'popular names', 'popular', 'most popular names', 'Most Popular Names Around the World', 'around the world', 'girl names', 'woman names', 'man namens', 'chinese names', 'top names', 'most common names', 'the infographics show', 'theinfographicsshow', 'popular name', 'most popular', 'top name', 'most common name', 'mon common indian name', 'most common male name', 'male name', 'female name'])
(ObjectId('601d0ba534fe727d22f1f098'), ['survive in space', 'space', 'tardigrades', 'tardigrade', 'water bear', 'moss piglet', 'bacteria', 'cannibal', 'mass extinction', 'radiation', 'oxygen depletion', 'pressure', 'cryptobiosis', 'metabolic process', 'International Society of Tardigrade Hunters', 'how to find a tardigrade', 'how to see a tardigrade', 'weird science', 'Low Earth Orbit', 'life noggin', 'science', 'animation', 'animated science', 'blocko'])
(ObjectId('601d0ba534fe727d

(ObjectId('601d0ba534fe727d22f1f1e2'), ['cars', 'climbkhana', 'gymkhana', 'drift', 'drifting', 'hoonigan', 'toyo tires', 'toyotires', 'pikespeak', 'pikes peak', 'racing', 'car', 'new', 'viral', 'google', 'instagram', 'facebook', 'hoonicornv2', 'hoonicorn', 'canyon', 'touge', 'how to', 'howto', 'diy', 'slide or dai', 'formula drift', 'formula d', 'ken block', 'kenblock', 'youtube', 'sema', 'methanol', 'turbo', 'boost', 'ford', 'mustang', 'classic car', 'classic cars', 'hooning', 'burn out', 'kill all tires', 'slay tires', 'sideways', 'how to burn out', 'burnout', 'dji', 'drone', 'films', 'video', 'photography', 'colorado', 'R888r', 'proxes', 'fifteen 52', 'tires'])
(ObjectId('601d0ba534fe727d22f1f1e3'), ['audi', 'audi tt rs', 'tt rs', 'think faster', 'audi think faster', 'adam scott', 'elizabeth banks', 'reddit', 'audi reddit', 'audi sport', 'audi sport reddit', '2018 audi tt rs', 'audi usa', 'audi of america', 'luxury', 'automotive', 'luxury vehicle', 'luxury automotive', 'audi ttrs', 

(ObjectId('601d0ba534fe727d22f1f324'), ['the food ranger', 'trevor james', 'food ranger', 'food', 'street food', 'street food around the world', 'street food videos', 'best', 'best street food', '2017', 'street', 'asian', 'try', 'chinese food', 'around the world', 'food street', 'cooking', 'chinese street food', 'world', 'china', 'chinese', 'spicy', 'sichuan', 'tour', 'street food in china', 'sichuan food', 'china street food', 'street food china', 'chinese cooking', 'street food (cuisine)', '中国菜', 'cheap', 'travel', 'chinese food (cuisine)', 'hot', 'vlog', 'top', 'street food 2017', 'china food', 'kitchen'])
(ObjectId('601d0ba534fe727d22f1f325'), ['Google', 'Pixel 2', 'Squeeze', 'Google Assistant', 'Phones', 'CNET News Video', 'John Kim'])
(ObjectId('601d0ba534fe727d22f1f326'), ['Google', 'Pixel 2', 'Squeeze', 'Google Assistant', 'Phones', 'CNET News Video', 'John Kim'])


In [23]:
cur_ex3final = collection_yt.find().limit(3)

In [24]:
list(cur_ex3final)

[{'_id': ObjectId('601d0ba534fe727d22f1d3ef'),
  'video_id': 'XpVt6Z1Gjjo',
  'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
  'channel_title': 'Logan Paul Vlogs',
  'category_id': 24,
  'tags': ['logan paul vlog',
   'logan paul',
   'logan',
   'paul',
   'olympics',
   'logan paul youtube',
   'vlog',
   'daily',
   'comedy',
   'hollywood',
   'parrot',
   'maverick',
   'bird',
   'maverick clothes',
   'diamond play button',
   'logan paul diamond play button',
   '10M subscribers',
   'logan paul 1 year vlogging',
   '1 year vlog',
   'dwarf mamba play button',
   'logan paul history',
   'youtube history',
   '10M',
   '10M plaque',
   'youtube button',
   'diamond button',
   'logang',
   'logang 4 life'],
  'views': 4394029,
  'likes': 320053,
  'dislikes': 5931,
  'comment_total': 46245,
  'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
  'date': 13.09,
  'snippet_title': 'Entertainment',
  'snippet_assignable': True},
 {'_id': 

## Question 4

In [25]:
cur_ex4 = collection_yt.find().sort([("views", -1)]).limit(10)

In [26]:
list(cur_ex4)

[{'_id': ObjectId('601d0ba534fe727d22f1e8b1'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09,
  'snippet_title': 'Music',
  'snippet_assignable': True},
 {'_id': ObjectId('601d0ba534fe727d22f1e89d'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09,
  'snippet_title': 'Music',
  'snippet_assignable': True},
 {'_id': ObjectId('601d0ba534fe727d22f1e788'),
  'video_id': 'tt

## Question 5

In [27]:
cur_ex5 = collection_yt.aggregate([{'$group' : {'_id': '$snippet_title', 'count_By_Category' : {'$sum' : 1}}}])

In [28]:
list(cur_ex5)

[{'_id': 'Gaming', 'count_By_Category': 82},
 {'_id': 'Pets & Animals', 'count_By_Category': 116},
 {'_id': 'Shows', 'count_By_Category': 2},
 {'_id': 'Entertainment', 'count_By_Category': 1601},
 {'_id': 'Science & Technology', 'count_By_Category': 512},
 {'_id': 'Music', 'count_By_Category': 1250},
 {'_id': 'Comedy', 'count_By_Category': 755},
 {'_id': 'Autos & Vehicles', 'count_By_Category': 116},
 {'_id': 'Howto & Style', 'count_By_Category': 869},
 {'_id': 'News & Politics', 'count_By_Category': 623},
 {'_id': 'People & Blogs', 'count_By_Category': 882},
 {'_id': 'Sports', 'count_By_Category': 410},
 {'_id': 'Nonprofits & Activism', 'count_By_Category': 14},
 {'_id': 'Travel & Events', 'count_By_Category': 48},
 {'_id': 'Film & Animation', 'count_By_Category': 378},
 {'_id': 'Education', 'count_By_Category': 334}]

## Question 6 

In [29]:
ex_cur6 = collection_yt.aggregate([
  {'$group': {
      '_id': "$channel_title",
      'avg_likes': {'$avg': "$likes"}
  }},
  {'$sort':{'avg_likes':-1}},
  {'$limit':10}  
    ])

In [30]:
list(ex_cur6)

[{'_id': 'ZaynVEVO', 'avg_likes': 1431683.0},
 {'_id': 'ibighit', 'avg_likes': 1371766.25},
 {'_id': 'melanie martinez', 'avg_likes': 911871.5},
 {'_id': 'BETNetworks', 'avg_likes': 769687.625},
 {'_id': 'jypentertainment', 'avg_likes': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'avg_likes': 653443.5384615385},
 {'_id': 'Wengie', 'avg_likes': 567235.2},
 {'_id': 'Alesso', 'avg_likes': 509417.28571428574},
 {'_id': 'PewDiePie', 'avg_likes': 508647.8},
 {'_id': 'shakiraVEVO', 'avg_likes': 434302.14285714284}]